In [1]:
!pip install mlxtend pyfpgrowth pandas

You should consider upgrading via the '/Users/komalagrawal/.virtualenvs/python3_vir/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from csv import reader
from collections import defaultdict
from itertools import combinations, chain
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import pyfpgrowth
import time

In [3]:
def run_apriori():
    dataset_choice = input("choose dataset ( amazon, bestbuy, kmart, nike, generic): ")
    # User input for minimum support and confidence
    support = float(input("Enter minimum support: "))
    confidence = float(input("Enter minimum confidence: "))
    # Reading the file and initializing sets
    with open(dataset_choice + "_transactions.csv", 'r') as file:
        csv_reader = reader(file)
        item_set = set()
        item_sets = []
        for line in csv_reader:
            cleaned_line = [value for value in line if value]
            record_set = set(cleaned_line)
            for item in record_set:
                item_set.add(frozenset([item]))
            item_sets.append(record_set)
    
    start_time_brute_force = time.time()
    
    # Initializing variables for the main algorithm
    global_freq_item_set = dict()
    global_item_set_with_sup = defaultdict(int)
    
    # getting items above minimum support
    freq_item_set = set()
    local_item_set_with_sup = defaultdict(int)
    
    for item in item_set:
        for item_set in item_sets:
            if item.issubset(item_set):
                global_item_set_with_sup[item] += 1
                local_item_set_with_sup[item] += 1
    
    for item, sup_count in local_item_set_with_sup.items():
        support_value = float(sup_count / len(item_sets))
        if support_value >= support:
            freq_item_set.add(item)
    
    L1_item_set = freq_item_set
    current_lset = L1_item_set
    k = 2
    
    while current_lset:
        global_freq_item_set[k - 1] = current_lset
        # Self-joining Lk
        candidate_set = set([i.union(j) for i in current_lset for j in current_lset if len(i.union(j)) == k])
    
        # Pruning
        temp_candidate_set = candidate_set.copy()
        for item in candidate_set:
            subsets = combinations(item, k - 1)
            for subset in subsets:
                if frozenset(subset) not in current_lset:
                    temp_candidate_set.remove(item)
                    break
        candidate_set = temp_candidate_set
    
        # Counting support again
        current_lset = set()
        local_item_set_with_sup = defaultdict(int)
        for item in candidate_set:
            for item_set in item_sets:
                if item.issubset(item_set):
                    global_item_set_with_sup[item] += 1
                    local_item_set_with_sup[item] += 1
    
        for item, sup_count in local_item_set_with_sup.items():
            support_value = float(sup_count / len(item_sets))
            if support_value >= support:
                current_lset.add(item)
        k += 1
    
    # Association rule generation
    rules = []
    for k, item_set in global_freq_item_set.items():
        for item in item_set:
            subsets = chain.from_iterable(combinations(item, r) for r in range(1, len(item)))
            for s in subsets:
                confidence_value = float(global_item_set_with_sup[item] / global_item_set_with_sup[frozenset(s)])
                if confidence_value > confidence:
                    rules.append([set(s), set(item.difference(s)), confidence_value])
    
    rules.sort(key=lambda x: x[2])
    
    end_time_brute_force = time.time()
    print(f"Brute-Force Time: {end_time_brute_force - start_time_brute_force} seconds")
    # Printing the results
    print("Frequent item sets")
    print(global_freq_item_set)
    
    print("Associated Rules:")
    for rule in rules:
        print(rule)
    
    print("--Applying Apriori Algo from a python library--")
    # Apriori Alogirm from mlxtend
    # Preprocess data for mlxtend
    start_time_apriori = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(item_sets).transform(item_sets)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    # Apply Apriori
    frequent_item_sets_apriori = apriori(df, min_support=support, use_colnames=True)
    
    # Generate rules
    rules_apriori = association_rules(frequent_item_sets_apriori, metric="confidence", min_threshold=confidence)
    end_time_apriori = time.time()
    print(f"Apriori Time: {end_time_apriori - start_time_apriori} seconds")
    
    # Print Apriori generated rules
    print("Apriori Rules")
    print(rules_apriori[['antecedents', 'consequents', 'support', 'confidence']])
    
    
    print("--Applying FP-Growth Algorith--")
    # FP growth algos
    # Find frequent patterns
    start_time_fpgrowth = time.time()
    patterns = pyfpgrowth.find_frequent_patterns(item_sets, support_threshold=int(support * len(item_sets)))
    
    # Generate association rules
    rules_fpgrowth = pyfpgrowth.generate_association_rules(patterns, confidence)
    end_time_fpgrowth = time.time()
    print(f"FP-Growth Time: {end_time_fpgrowth - start_time_fpgrowth} seconds")
    
    # Print FP-Growth generated rules
    print("FP-Growth Rules")
    for rule, metrics in rules_fpgrowth.items():
        print(f"Antecedent: {rule}, Consequent: {metrics[0]}, Confidence: {metrics[1]}")

In [4]:
# running amazon dataset with 0.4 support and 0.4 confidence
run_apriori()

choose dataset ( amazon, bestbuy, kmart, nike, generic):  amazon
Enter minimum support:  0.4
Enter minimum confidence:  0.4


Brute-Force Time: 0.00033783912658691406 seconds
Frequent item sets
{1: {frozenset({' Android Programming: The Big Nerd Ranch'}), frozenset({' Java For Dummies'}), frozenset({'A Beginner’s Guide'}), frozenset({' Java: The Complete Reference'})}, 2: {frozenset({' Java For Dummies', 'A Beginner’s Guide'}), frozenset({'A Beginner’s Guide', ' Java: The Complete Reference'}), frozenset({' Java For Dummies', ' Java: The Complete Reference'})}, 3: {frozenset({' Java For Dummies', 'A Beginner’s Guide', ' Java: The Complete Reference'})}}
Associated Rules:
[{'A Beginner’s Guide'}, {' Java For Dummies'}, 0.8181818181818182]
[{'A Beginner’s Guide'}, {' Java: The Complete Reference'}, 0.8181818181818182]
[{'A Beginner’s Guide'}, {' Java For Dummies', ' Java: The Complete Reference'}, 0.8181818181818182]
[{' Java For Dummies'}, {'A Beginner’s Guide'}, 0.9]
[{' Java For Dummies'}, {' Java: The Complete Reference'}, 0.9]
[{' Java For Dummies'}, {'A Beginner’s Guide', ' Java: The Complete Reference'},

In [5]:
# running amazon dataset with 0.3 support and 0.3 confidence
run_apriori()

choose dataset ( amazon, bestbuy, kmart, nike, generic):  amazon
Enter minimum support:  0.3
Enter minimum confidence:  0.3


Brute-Force Time: 0.00025200843811035156 seconds
Frequent item sets
{1: {frozenset({' Android Programming: The Big Nerd Ranch'}), frozenset({' Java For Dummies'}), frozenset({'A Beginner’s Guide'}), frozenset({' Java: The Complete Reference'})}, 2: {frozenset({' Java For Dummies', 'A Beginner’s Guide'}), frozenset({'A Beginner’s Guide', ' Java: The Complete Reference'}), frozenset({' Java For Dummies', ' Java: The Complete Reference'})}, 3: {frozenset({' Java For Dummies', 'A Beginner’s Guide', ' Java: The Complete Reference'})}}
Associated Rules:
[{'A Beginner’s Guide'}, {' Java For Dummies'}, 0.8181818181818182]
[{'A Beginner’s Guide'}, {' Java: The Complete Reference'}, 0.8181818181818182]
[{'A Beginner’s Guide'}, {' Java For Dummies', ' Java: The Complete Reference'}, 0.8181818181818182]
[{' Java For Dummies'}, {'A Beginner’s Guide'}, 0.9]
[{' Java For Dummies'}, {' Java: The Complete Reference'}, 0.9]
[{' Java For Dummies'}, {'A Beginner’s Guide', ' Java: The Complete Reference'},

In [7]:
# running bestbuy dataset with 0.4 support and 0.4 confidence
run_apriori()

choose dataset ( amazon, bestbuy, kmart, nike, generic):  bestbuy
Enter minimum support:  0.4
Enter minimum confidence:  0.4


Brute-Force Time: 0.0009181499481201172 seconds
Frequent item sets
{1: {frozenset({' Flash Drive'}), frozenset({' Speakers'}), frozenset({' External Hard-Drive'}), frozenset({' Printer'}), frozenset({' Lab Top Case'}), frozenset({' Anti-Virus'}), frozenset({' Microsoft Office'}), frozenset({'Digital Camera '})}, 2: {frozenset({' Speakers', ' Anti-Virus'}), frozenset({' Flash Drive', ' Printer'}), frozenset({' Speakers', ' Lab Top Case'}), frozenset({' Microsoft Office', ' Anti-Virus'}), frozenset({' Flash Drive', ' Lab Top Case'}), frozenset({' Flash Drive', ' Microsoft Office'}), frozenset({' Lab Top Case', ' External Hard-Drive'}), frozenset({' Anti-Virus', ' External Hard-Drive'}), frozenset({' Microsoft Office', ' Printer'}), frozenset({' Lab Top Case', ' Anti-Virus'}), frozenset({' Flash Drive', ' Anti-Virus'})}, 3: {frozenset({' Flash Drive', ' Microsoft Office', ' Anti-Virus'}), frozenset({' Lab Top Case', ' Anti-Virus', ' External Hard-Drive'}), frozenset({' Flash Drive', ' Mic

In [8]:
# running kmart dataset with 0.4 support and 0.4 confidence
run_apriori()

choose dataset ( amazon, bestbuy, kmart, nike, generic):  kmart
Enter minimum support:  0.4
Enter minimum confidence:  0.4


Brute-Force Time: 0.0002796649932861328 seconds
Frequent item sets
{1: {frozenset({' Kids Bedding'}), frozenset({'Decorative Pillows'}), frozenset({' Shams'}), frozenset({' Bed Skirts'})}, 2: {frozenset({' Kids Bedding', ' Bed Skirts'}), frozenset({' Bed Skirts', ' Shams'})}}
Associated Rules:
[{' Bed Skirts'}, {' Kids Bedding'}, 0.7272727272727273]
[{' Bed Skirts'}, {' Shams'}, 0.7272727272727273]
[{' Kids Bedding'}, {' Bed Skirts'}, 0.8]
[{' Shams'}, {' Bed Skirts'}, 0.8]
--Applying Apriori Algo from a python library--
Apriori Time: 0.008028984069824219 seconds
Apriori Rules
       antecedents      consequents  support  confidence
0  ( Kids Bedding)    ( Bed Skirts)      0.4    0.800000
1    ( Bed Skirts)  ( Kids Bedding)      0.4    0.727273
2    ( Bed Skirts)         ( Shams)      0.4    0.727273
3         ( Shams)    ( Bed Skirts)      0.4    0.800000
--Applying FP-Growth Algorith--
FP-Growth Time: 0.00044798851013183594 seconds
FP-Growth Rules
Antecedent: (' Bed Skirts',), Conseq

In [12]:
# running nike dataset with 0.4 support and 0.4 confidence
run_apriori()

choose dataset ( amazon, bestbuy, kmart, nike, generic):  nike
Enter minimum support:  0.4
Enter minimum confidence:  0.4


Brute-Force Time: 0.0006840229034423828 seconds
Frequent item sets
{1: {frozenset({' Modern Pants'}), frozenset({' Sweatshirts'}), frozenset({' Socks'}), frozenset({' Rash Guard'}), frozenset({'Running Shoe'}), frozenset({' Tech Pants'}), frozenset({' Dry Fit V-Nick'})}, 2: {frozenset({' Sweatshirts', ' Modern Pants'}), frozenset({' Socks', 'Running Shoe'}), frozenset({' Modern Pants', 'Running Shoe'}), frozenset({' Tech Pants', ' Rash Guard'}), frozenset({' Sweatshirts', ' Socks'}), frozenset({' Sweatshirts', 'Running Shoe'}), frozenset({' Socks', ' Modern Pants'}), frozenset({' Rash Guard', ' Dry Fit V-Nick'})}, 3: {frozenset({' Socks', ' Modern Pants', 'Running Shoe'}), frozenset({' Socks', 'Running Shoe', ' Sweatshirts'}), frozenset({' Sweatshirts', ' Socks', ' Modern Pants'}), frozenset({' Sweatshirts', ' Modern Pants', 'Running Shoe'})}, 4: {frozenset({' Socks', ' Modern Pants', 'Running Shoe', ' Sweatshirts'})}}
Associated Rules:
[{'Running Shoe'}, {' Socks', ' Modern Pants'}, 0

In [11]:
# running generic dataset with 0.3 support and 0.3 confidence
run_apriori()

choose dataset ( amazon, bestbuy, kmart, nike, generic):  generic
Enter minimum support:  0.3
Enter minimum confidence:  0.3


Brute-Force Time: 0.00040912628173828125 seconds
Frequent item sets
{1: {frozenset({'A'}), frozenset({'E'}), frozenset({'D'}), frozenset({'H'}), frozenset({'C'}), frozenset({'G'}), frozenset({'B'}), frozenset({'F'})}, 2: {frozenset({'A', 'E'})}}
Associated Rules:
[{'A'}, {'E'}, 0.5454545454545454]
[{'E'}, {'A'}, 0.6666666666666666]
--Applying Apriori Algo from a python library--
Apriori Time: 0.01174473762512207 seconds
Apriori Rules
  antecedents consequents  support  confidence
0         (A)         (E)      0.3    0.545455
1         (E)         (A)      0.3    0.666667
--Applying FP-Growth Algorith--
FP-Growth Time: 0.0006399154663085938 seconds
FP-Growth Rules
Antecedent: ('A',), Consequent: ('E',), Confidence: 0.5454545454545454
Antecedent: ('E',), Consequent: ('A',), Confidence: 0.6666666666666666
